In [133]:
#%run /home/sjk/projects/jupyter_notebooks/scripts/mysql_connection_fireducks.py
%run /home/sjk/projects/jupyter_notebooks/scripts/mysql_connection_aws.py

%matplotlib inline
%pylab inline

# graph defaults
import matplotlib.style
import matplotlib as mpl
#mpl.style.use('classic')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
mpl.rcParams['figure.facecolor'] = '1'
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

mpl.rcParams['figure.figsize'] = [8.0, 4.0]
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 8
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['figure.titlesize'] = 'medium'

mpl.rcParams['lines.linewidth'] = 0.6

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [132]:
# LOAD DATA FROM research.strategy_returns
strat_num = 112
report_file = "sg_commodity_basket_report.html"

sql_str = "select * from research.strategy_returns where strategy_number = "+str(strat_num)+" order by trade_date"
df_returns = LoadSqlData(sql_str, conn_colo)

df_returns['YYYY_MM'] = [str(x.year) + '_' + '{:02}'.format(x.month) for x in df_returns['trade_date'].values]

df_monthly = df_returns.groupby('YYYY_MM')['return_value'].sum()
df_monthly = pd.DataFrame({
    'YYYY_MM': df_monthly.index,
    'monthly_return': df_monthly.values
})

sql_str = "select strategy_description from research.strategy_details where strategy_number="+str(strat_num)
df_desc = LoadSqlData(sql_str, conn_colo)
strat_desc = df_desc['strategy_description'].values[0]
print(strat_desc)


# CREATE REPORT

from datetime import datetime

print(f"\nTotal records: {len(df_monthly)}")

# Generate the report
report_file = generate_monthly_returns_report(
    df_monthly, 
    strategy_name= strat_desc,
    output_file=report_file
)

print(f"\nReport saved as: {report_file}")
print("Open the HTML file in your web browser to view the report.")


SocGen Commodity Basket Solutions - BCOM Hedged

Total records: 255
Report generated successfully: sg_commodity_basket_report.html

Report saved as: sg_commodity_basket_report.html
Open the HTML file in your web browser to view the report.


In [123]:
df_monthly

,YYYY_MM,monthly_return
0,2016_07,0.01894
1,2016_08,0.02006
2,2016_09,0.01374
3,2016_10,0.02111
4,2016_11,0.03487
...,...,...
103,2025_02,0.330255
104,2025_03,0.36432
105,2025_04,0.353067
106,2025_05,0.369173


In [136]:
# LOAD DATA FROM research.sim_results
sql_str = "select * from research.sim_results where run_number = 138 order by trade_date"
df_returns = LoadSqlData(sql_str, conn_colo)

df_returns['YYYY_MM'] = [str(x.year) + '_' + '{:02}'.format(x.month) for x in df_returns['trade_date'].values]

df_monthly = df_returns.groupby('YYYY_MM')['dailyPnL'].sum()
df_monthly = pd.DataFrame({
    'YYYY_MM': df_monthly.index,
    'monthly_return': 100.0*df_monthly.values
})

In [137]:
# CREATE REPORT

from datetime import datetime

print(f"\nTotal records: {len(df_monthly)}")

# Generate the report
report_file = generate_monthly_returns_report(
    df_monthly, 
    strategy_name="SPX 2-year ATM LDC Returns",
    output_file="spx_ldc_100K_report.html"
)

print(f"\nReport saved as: {report_file}")
print("Open the HTML file in your web browser to view the report.")



Total records: 357
Report generated successfully: spx_ldc_100K_report.html

Report saved as: spx_ldc_100K_report.html
Open the HTML file in your web browser to view the report.


### Data Manipulation for Return Stats

In [34]:
 data = df_monthly.copy()
    
# Parse the YYYY_MM column to extract year and month
data['year'] = data['YYYY_MM'].astype(str).str[:4].astype(int)
data['month'] = data['YYYY_MM'].astype(str).str[5:].astype(int)

# Create pivot table with years as index and months as columns
pivot_table = data.pivot(index='year', columns='month', values='monthly_return')

# Reorder columns to show months 1-12
month_cols = [i for i in range(1, 13) if i in pivot_table.columns]
pivot_table = pivot_table[month_cols]

# Calculate annual returns (sum of monthly returns for each year)
pivot_table['Annual Total'] = pivot_table.sum(axis=1)


In [44]:
(pivot_table[month_cols]/100.0 + 1).prod(axis=1)-1

year
2005    0.069214
2006    0.194139
2007    0.119758
2008   -0.269612
2009    0.295522
2010    0.147549
2011    0.002853
2012    0.204826
2013    0.229938
2014    0.076407
2015   -0.006475
2016    0.042256
2017    0.284176
2018   -0.088142
2019    0.170470
2020    0.007938
2021    0.288316
2022   -0.174933
2023    0.150378
2024    0.211085
2025    0.106207
dtype: float64

In [51]:
annual_returns = pivot_table['Annual Total']
monthly_returns = data['monthly_return']

best_year = annual_returns.idxmax()
worst_year = annual_returns.idxmin()
avg_annual_return = annual_returns.mean()
total_return = annual_returns.sum()


In [57]:
pow((annual_returns/100.0 + 1).prod(),(1.0/(len(monthly_returns)/12)))-1.0

0.08328514610016491

In [60]:
pow((monthly_returns/100.0 + 1).prod(),(1.0/(len(monthly_returns)/12)))-1.0

0.08895803294980076

In [62]:
(monthly_returns/100.0 + 1).prod() - 1.0

4.819651169428067

#### GENERATE REPORTS

In [98]:
import pandas as pd
import numpy as np
from datetime import datetime

def generate_monthly_returns_report(df, strategy_name="Investment Strategy", output_file="monthly_returns_report.html"):
    """
    Generate a web report of monthly return data in matrix format.
    
    Parameters:
    df (pd.DataFrame): DataFrame with columns 'YYYY_MM' and 'monthly_return'
    strategy_name (str): Name of the investment strategy for the report title
    output_file (str): Output HTML file name
    
    Returns:
    str: Path to the generated HTML file
    """
    
    # Create a copy of the dataframe to avoid modifying the original
    data = df.copy()
    
    # Parse the YYYY_MM column to extract year and month
    data['year'] = data['YYYY_MM'].astype(str).str[:4].astype(int)
    data['month'] = data['YYYY_MM'].astype(str).str[5:].astype(int)
    
    # Create pivot table with years as index and months as columns
    pivot_table = data.pivot(index='year', columns='month', values='monthly_return')
    
    # Reorder columns to show months 1-12
    month_cols = [i for i in range(1, 13) if i in pivot_table.columns]
    pivot_table = pivot_table[month_cols]
    
    # Calculate annual returns (sum of monthly returns for each year)
    #pivot_table['Annual Total'] = pivot_table.sum(axis=1)
    # Use Geometric Returns
    pivot_table['Annual Total'] = 100.0*((pivot_table[month_cols]/100.0 + 1).prod(axis=1)-1.0)

    
    # Prepare data for charts
    # Cumulative returns data
    cumulative_data = data.sort_values(['year', 'month']).copy()
    cumulative_data['cumulative_return'] = (1 + cumulative_data['monthly_return'] / 100).cumprod() - 1.0
    cumulative_data['date_label'] = cumulative_data['YYYY_MM'].str.replace('_', '-')
    
    # Convert to lists for JavaScript
    cumulative_labels = cumulative_data['date_label'].tolist()
    cumulative_values = [round(val * 100, 2) for val in cumulative_data['cumulative_return'].tolist()]    
    
    # Annual returns data
    annual_data = pivot_table['Annual Total'].to_dict()
    annual_years = list(annual_data.keys())
    annual_values = [round(val, 2) for val in annual_data.values()]
    annual_colors_bg = ['#27ae60' if v >= 0 else '#e74c3c' for v in annual_values]
    annual_colors_border = ['#1e8449' if v >= 0 else '#c0392b' for v in annual_values]
    
    # Monthly returns histogram data
    monthly_bins = [-10, -5, -3, -1, 0, 1, 3, 5, 10, 15]
    monthly_hist, _ = np.histogram(data['monthly_return'], bins=monthly_bins)
    monthly_bin_labels = [f'{monthly_bins[i]:.0f}% to {monthly_bins[i+1]:.0f}%' for i in range(len(monthly_bins)-1)]
    monthly_hist_values = monthly_hist.tolist()
    
    # Generate HTML content
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>{strategy_name} - Monthly Returns Report</title>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.9.1/chart.min.js"></script>
        <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 20px;
                background-color: #f5f5f5;
            }}
            
            .container {{
                max-width: 1200px;
                margin: 0 auto;
                background-color: white;
                padding: 30px;
                border-radius: 8px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}
            
            h1 {{
                color: #2c3e50;
                text-align: center;
                margin-bottom: 10px;
                font-size: 2.2em;
            }}
            
            .subtitle {{
                text-align: center;
                color: #7f8c8d;
                margin-bottom: 30px;
                font-size: 1.1em;
            }}
            
            .report-table {{
                width: 100%;
                border-collapse: collapse;
                margin: 20px 0;
                font-size: 0.9em;
            }}
            
            .report-table th {{
                background-color: #34495e;
                color: white;
                padding: 12px 8px;
                text-align: center;
                font-weight: bold;
                border: 1px solid #2c3e50;
            }}
            
            .report-table td {{
                padding: 8px;
                text-align: center;
                border: 1px solid #bdc3c7;
            }}
            
            .year-header {{
                background-color: #ecf0f1;
                font-weight: bold;
                color: #2c3e50;
            }}
            
            .positive {{
                background-color: #d5f4e6;
                color: #27ae60;
            }}
            
            .negative {{
                background-color: #fadbd8;
                color: #e74c3c;
            }}
            
            .annual-total {{
                background-color: #f8f9fa;
                font-weight: bold;
                border-left: 3px solid #3498db;
            }}
            
            .summary {{
                margin-top: 30px;
                padding: 20px;
                background-color: #f8f9fa;
                border-radius: 5px;
                border-left: 4px solid #3498db;
            }}
            
            .summary h3 {{
                color: #2c3e50;
                margin-top: 0;
            }}
            
            .summary-stats {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 15px;
                margin-top: 15px;
            }}
            
            .stat-item {{
                text-align: center;
                padding: 10px;
                background-color: white;
                border-radius: 5px;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            }}
            
            .stat-value {{
                font-size: 1.5em;
                font-weight: bold;
                color: #2c3e50;
            }}
            
            .stat-label {{
                color: #7f8c8d;
                font-size: 0.9em;
            }}
            
            .charts-section {{
                margin-top: 40px;
            }}
            
            .chart-container {{
                margin: 30px 0;
                padding: 20px;
                background-color: white;
                border-radius: 8px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}
            
            .chart-title {{
                color: #2c3e50;
                font-size: 1.4em;
                margin-bottom: 20px;
                text-align: center;
                font-weight: bold;
            }}
            
            .chart-canvas {{
                max-height: 400px;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>{strategy_name}</h1>
            <div class="subtitle">Monthly Returns Report - Generated on {datetime.now().strftime('%B %d, %Y')}</div>
            
            <table class="report-table">
                <thead>
                    <tr>
                        <th>Year</th>
                        <th>Jan</th>
                        <th>Feb</th>
                        <th>Mar</th>
                        <th>Apr</th>
                        <th>May</th>
                        <th>Jun</th>
                        <th>Jul</th>
                        <th>Aug</th>
                        <th>Sep</th>
                        <th>Oct</th>
                        <th>Nov</th>
                        <th>Dec</th>
                        <th>Annual Total</th>
                    </tr>
                </thead>
                <tbody>
    """
    
    # Add table rows for each year
    for year in sorted(pivot_table.index):
        html_content += f'                    <tr>\n                        <td class="year-header">{year}</td>\n'
        
        # Add monthly returns
        for month in range(1, 13):
            value = pivot_table.loc[year, month] if month in pivot_table.columns and not pd.isna(pivot_table.loc[year, month]) else None
            
            if value is not None:
                # Format the value as percentage
                formatted_value = f"{value:.2f}%"
                css_class = "positive" if value >= 0 else "negative"
                html_content += f'                        <td class="{css_class}">{formatted_value}</td>\n'
            else:
                html_content += '                        <td>-</td>\n'
        
        # Add annual total
        annual_total = pivot_table.loc[year, 'Annual Total']
        annual_formatted = f"{annual_total:.2f}%"
        annual_class = "annual-total positive" if annual_total >= 0 else "annual-total negative"
        html_content += f'                        <td class="{annual_class}">{annual_formatted}</td>\n'
        html_content += '                    </tr>\n'
    
    # Calculate summary statistics
    annual_returns = pivot_table['Annual Total']
    monthly_returns = data['monthly_return']

    best_year = annual_returns.idxmax()
    worst_year = annual_returns.idxmin()
    
    #avg_annual_return = annual_returns.mean()
    # calculate average annual return as average geometric annual return, based on monthly compounding
    avg_annual_return = 100.0*(pow((monthly_returns/100.0 + 1).prod(),(1.0/(len(monthly_returns)/12)))-1.0)
    #total_return = annual_returns.sum()
    # calculate total return as compounded monthly return - 1.0
    total_return = 100.0*((monthly_returns/100.0 + 1).prod() - 1.0)
    
    html_content += f"""
                </tbody>
            </table>
            
            <div class="summary">
                <h3>Summary Statistics</h3>
                <div class="summary-stats">
                    <div class="stat-item">
                        <div class="stat-value">{best_year}</div>
                        <div class="stat-label">Best Year</div>
                        <div class="stat-label">({annual_returns.loc[best_year]:.2f}%)</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{worst_year}</div>
                        <div class="stat-label">Worst Year</div>
                        <div class="stat-label">({annual_returns.loc[worst_year]:.2f}%)</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{avg_annual_return:.2f}%</div>
                        <div class="stat-label">Average Annual Return</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{total_return:.2f}%</div>
                        <div class="stat-label">Total Cumulative Return</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{len(annual_returns)}</div>
                        <div class="stat-label">Years of Data</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value">{len(monthly_returns)}</div>
                        <div class="stat-label">Months of Data</div>
                    </div>
                </div>
            </div>
            
            <div class="charts-section">
                <div class="chart-container">
                    <div class="chart-title">Cumulative Returns Over Time</div>
                    <canvas id="cumulativeChart" class="chart-canvas"></canvas>
                </div>
                
                <div class="chart-container">
                    <div class="chart-title">Annual Returns</div>
                    <canvas id="annualChart" class="chart-canvas"></canvas>
                </div>
                
                <div class="chart-container">
                    <div class="chart-title">Monthly Returns Distribution</div>
                    <canvas id="monthlyHistogram" class="chart-canvas"></canvas>
                </div>
            </div>
        </div>
        
        <script>
            // Cumulative Returns Chart
            const cumulativeCtx = document.getElementById('cumulativeChart').getContext('2d');
            const cumulativeChart = new Chart(cumulativeCtx, {{
                type: 'line',
                data: {{
                    labels: {cumulative_labels},
                    datasets: [{{
                        label: 'Cumulative Return (%)',
                        data: {cumulative_values},
                        borderColor: '#3498db',
                        backgroundColor: 'rgba(52, 152, 219, 0.1)',
                        borderWidth: 2,
                        fill: true,
                        tension: 0.3
                    }}]
                }},
                options: {{
                    responsive: true,
                    maintainAspectRatio: false,
                    scales: {{
                        y: {{
                            title: {{
                                display: true,
                                text: 'Cumulative Return (%)'
                            }},
                            grid: {{
                                color: 'rgba(0,0,0,0.1)'
                            }}
                        }},
                        x: {{
                            title: {{
                                display: true,
                                text: 'Date'
                            }},
                            grid: {{
                                color: 'rgba(0,0,0,0.1)'
                            }}
                        }}
                    }},
                    plugins: {{
                        legend: {{
                            display: true,
                            position: 'top'
                        }}
                    }}
                }}
            }});
            
            // Annual Returns Chart
            const annualCtx = document.getElementById('annualChart').getContext('2d');
            const annualChart = new Chart(annualCtx, {{
                type: 'bar',
                data: {{
                    labels: {annual_years},
                    datasets: [{{
                        label: 'Annual Return (%)',
                        data: {annual_values},
                        backgroundColor: {annual_colors_bg},
                        borderColor: {annual_colors_border},
                        borderWidth: 1
                    }}]
                }},
                options: {{
                    responsive: true,
                    maintainAspectRatio: false,
                    scales: {{
                        y: {{
                            title: {{
                                display: true,
                                text: 'Annual Return (%)'
                            }},
                            grid: {{
                                color: 'rgba(0,0,0,0.1)'
                            }},
                            beginAtZero: true
                        }},
                        x: {{
                            title: {{
                                display: true,
                                text: 'Year'
                            }},
                            grid: {{
                                color: 'rgba(0,0,0,0.1)'
                            }}
                        }}
                    }},
                    plugins: {{
                        legend: {{
                            display: false
                        }}
                    }}
                }}
            }});
            
            // Monthly Returns Histogram
            const histogramCtx = document.getElementById('monthlyHistogram').getContext('2d');
            const histogramChart = new Chart(histogramCtx, {{
                type: 'bar',
                data: {{
                    labels: {monthly_bin_labels},
                    datasets: [{{
                        label: 'Frequency',
                        data: {monthly_hist_values},
                        backgroundColor: 'rgba(52, 152, 219, 0.6)',
                        borderColor: '#3498db',
                        borderWidth: 1
                    }}]
                }},
                options: {{
                    responsive: true,
                    maintainAspectRatio: false,
                    scales: {{
                        y: {{
                            title: {{
                                display: true,
                                text: 'Number of Months'
                            }},
                            grid: {{
                                color: 'rgba(0,0,0,0.1)'
                            }},
                            beginAtZero: true
                        }},
                        x: {{
                            title: {{
                                display: true,
                                text: 'Monthly Return Range'
                            }},
                            grid: {{
                                color: 'rgba(0,0,0,0.1)'
                            }}
                        }}
                    }},
                    plugins: {{
                        legend: {{
                            display: false
                        }}
                    }}
                }}
            }});
        </script>
    </body>
    </html>
    """
    
    # Write HTML to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"Report generated successfully: {output_file}")
    return output_file



In [89]:
# CREATE REPORT

from datetime import datetime

print(f"\nTotal records: {len(df_monthly)}")

# Generate the report
report_file = generate_monthly_returns_report(
    df_monthly, 
    strategy_name="ResEq LDC\Strangle All Combined Backtest",
    output_file="reseq_report.html"
)

print(f"\nReport saved as: {report_file}")
print("Open the HTML file in your web browser to view the report.")



Total records: 248
Report generated successfully: reseq_report.html

Report saved as: reseq_report.html
Open the HTML file in your web browser to view the report.
